In [1]:
import os
from glob import glob

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
train_dir = '/content/gdrive/My Drive/fastcampus/mnist_png/training'#모든 파일의 절대경로를 다 가져옴
test_dir = '/content/gdrive/My Drive/fastcampus/mnist_png/testing'

In [4]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.3,
    zoom_range = 0.2,
    horizontal_flip = True
)
test_datagen = ImageDataGenerator(
    rescale = 1./255#rescale은 그대로 놔둬라
)

In [5]:
num_epochs =10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28,28,1)
num_classes = 10

In [7]:
train_generator = train_datagen.flow_from_directory(#이렇게 생긴 구조는 개꿀
    train_dir,
    target_size = input_shape[:2],#channel은 안넣어도 됨 gray자너~
    batch_size = batch_size,
    color_mode='grayscale',
    class_mode = 'categorical'#binary면 바꿔주기
)

Found 60001 images belonging to 10 classes.


In [8]:
validation_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size = input_shape[:2],#channel은 안넣어도 됨 gray자너~
    batch_size = batch_size,
    color_mode='grayscale',
    class_mode = 'categorical'#binary면 바꿔주기
    #혹시 안되면 classes추가
)

Found 10000 images belonging to 10 classes.


In [9]:
inputs = layers.Input(shape = input_shape)
#feature extraction
net = layers.Conv2D(32,3,padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,3,padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64,3,padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,3,padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D((2,2))(net)
net = layers.Dropout(dropout_rate)(net)

#Fullcy Connected
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net, name = 'Basic_CNN')

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['Accuracy'])

#학습


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),#몇개의 batch를 할 거냐
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = len(validation_generator)
)#generator는 generator만들었던걸 넣어라

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
  44/1876 [..............................] - ETA: 4:01:19 - loss: 2.2991 - accuracy: 0.0000e+00